# Homework 3 of Machine Learning Zoomcamp
This homework is using lead scoring bank marketing dataset.
In this dataset our desired target for classification task will be __*converted*__ variable - has the client signed up to the platform or not.

In [2]:
#import all necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#downloading the data
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-13 17:38:47--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.03s   

2025-10-13 17:38:47 (2.31 MB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [3]:
#read the dataset and assign it to dataframe
df = pd.read_csv('course_lead_scoring.csv')

#checking the first 5 data
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


### Data preparation

    * Check if the missing values are presented in the features.
    * If there are missing values:
        - For caterogiral features, replace them with 'NA'
        - For numerical features, replace with with 0.0


In [4]:
#Checking if missing values exist
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
#checking types of data in each column
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
#replacing missing value of categorical features with NA
cat_cols = ['lead_source', 'industry', 'employment_status', 'location']
df[cat_cols] = df[cat_cols].fillna('NA')

#replacing missing values of numerical feature with 0.0
df['annual_income'] = df.annual_income.fillna(0.0)

#checking the missing values of the dataset
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1
What is the most frequent observation (mode) for the column industry?

In [10]:
#checking the mode of 'industry'
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [12]:
#interested to know the exact number of each value in industry
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

**Answer**: The most frequent observation for industry column is __*retail*__

### Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

   * __interaction_count__ and __lead_score__
   * __number_of_courses_viewed__ and __lead_score__
   * __number_of_courses_viewed__ and __interaction_count__
   * __annual_income__ and __interaction_count__

Only consider the pairs above when answering this question.

In [18]:
#finding out the correlation between interaction_count and number_of_courses_viewed with lead_score
df[['interaction_count', 'number_of_courses_viewed']].corrwith(df['lead_score'])

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [19]:
#finding out the correlation between number_of_courses_viewed and annual_income with interaction_count
df[['number_of_courses_viewed','annual_income']].corrwith(df['interaction_count'])

number_of_courses_viewed   -0.023565
annual_income               0.027036
dtype: float64

**Answer**: The features with the biggest correlation are __*annual_income*__ and __*interaction_count*__

## Split the data

   * Split your data in train/val/test sets with 60%/20%/20% distribution.
   * Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
   * Make sure that the target value y is not in your dataframe.


In [21]:
#importing function from skilearn module
from sklearn.model_selection import train_test_split

Please refer train_test_split to [this documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 

In [22]:
#we're going split data into 3 parts, so we'll use the train_test_split twice. The first one the train + val combined into full test
#and test set. The second one we split full test into train and validation.
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

#the test size below is 0.25 or 25% from df_full_train (which is 80% of overall df) that equals to 20% of overall df
#so 20% val / 80% full train = 1/4 = 25%
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [23]:
#check the length of all data
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [25]:
#reset the shuffled index of each part of dataset
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [29]:
#setting the target variable as a new 1D array
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values


In [31]:
#deleting target values in dataframe to avoid mistakes in the future
del df_train['converted']
del df_val['converted']
del df_test['converted']

### Question 3

    Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
    Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

   * industry
   * location
   * lead_source
   * employment_status


In [24]:
#importing from sklearn
from sklearn.metrics import mutual_info_score

Mutual information - concept from information theory, it tells us how much we can learn about one variable if we know the value of another.  
Read this:  
* https://en.wikipedia.org/wiki/Mutual_information
* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mutual_info_score.html

In [37]:
round(mutual_info_score(df_full_train.converted, df_full_train.industry), 2)

0.01

In [38]:
round(mutual_info_score(df_full_train.converted, df_full_train.location), 2)

0.0

In [39]:
round(mutual_info_score(df_full_train.converted, df_full_train.lead_source), 2)

0.03

In [40]:
round(mutual_info_score(df_full_train.converted, df_full_train.employment_status), 2)

0.01

**Answer**: The biggest mutual information score with converted is __*lead_source*__


### Question 4

   * Now let's train a logistic regression.
   * Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
   * Fit the model on the training dataset.
       - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
        model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
   * Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

    * 0.64
    * 0.74
    * 0.84
    * 0.94


#### One-hot encoding
Using scikit-learn to encode categorical features

In [41]:
#importing function from scikit
from sklearn.feature_extraction import DictVectorizer

In [42]:
#separating categorical and numerical data in our dataset. Numerical for column in int and float data types,
#categorical for column with object data type, except the desired target 'converted'
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry','employment_status', 'location']

Reference: 
* the DictVectorizer [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html)
* pd.DataFrame.to_dict [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)

In [45]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)

#### Logistic Regression

In [46]:
#importing function
from sklearn.linear_model import LogisticRegression

In [47]:
#fitting the model using suggested parameters
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [48]:
#create soft prediction using validation dataset. We only interested in probability of converted, that's why we take [:,1}
y_pred = model.predict_proba(X_val)[:,1]

In [50]:
#the converted probability is the one thatthe y_prediction >= 0.5
converted_decision = (y_pred >= 0.5)

In [51]:
#checking the accuracy of prediction. if y_val equals to converted_decision means True, otherwise False
accuracy = round((y_val == converted_decision).mean(), 2)
accuracy

np.float64(0.7)

__Answer__: the accuracy is __*0.7*__

### Question 5

   * Let's find the least useful feature using the feature elimination technique.
   * Train a model using the same features and parameters as in Q4 (without rounding).
   * Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
   * For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

    * 'industry'
    * 'employment_status'
    * 'lead_score'

    Note: The difference doesn't have to be positive.


In [60]:
#we separate the feature we're interested in from the original numerical and categorical feature
numerical_minus = ['number_of_courses_viewed', 'annual_income', 'interaction_count']
categorical_minus = ['lead_source', 'location']
feat_interest = ['industry', 'employment_status', 'lead_score']

for i in feat_interest:
    df_train_temp = df_train.drop(i, axis='columns') #removing specific labels using pandas .drop() for temporary train dataframe
    train_dict_temp = df_train_temp.to_dict(orient='records')
    dv_temp = DictVectorizer()
    X_train_temp = dv_temp.fit_transform(train_dict_temp)
    
    model.fit(X_train_temp, y_train) #the model and y_train are same as we use in the Question 4 Logistic Regression 
    
    df_val_temp = df_val.drop(i, axis='columns') #removing specific labels using pandas .drop() for temporary validation dataframe
    val_dict_temp = df_val_temp.to_dict(orient='records')
    X_val_temp = dv_temp.transform(val_dict_temp)

    #create soft prediction using validation dataset. We only interested in probability of converted, that's why we take [:,1}
    y_pred_temp = model.predict_proba(X_val_temp)[:, 1] #the model is same as we use in the Question 4 Logistic Regression
    
    converted_decision_temp = (y_pred_temp >= 0.5) #the converted probability is the one thatthe y_prediction >= 0.5

    #checking the accuracy of prediction. if y_val equals to converted_decision_temp means True, otherwise False
    accuracy_temp = (y_val == converted_decision_temp).mean()  

    diff_accuracy = accuracy - accuracy_temp #difference between original accuracy in question 4 and the accuracy without the feature.

    print(f'Accuracy difference between LogReg model with all features and without "{i}" feature: {diff_accuracy}')



Accuracy difference between LogReg model with all features and without "industry" feature: 0.0003412969283276279
Accuracy difference between LogReg model with all features and without "employment_status" feature: 0.003754266211604018
Accuracy difference between LogReg model with all features and without "lead_score" feature: -0.006484641638225264


__Answer__: As the difference doesn't have to be positive, so the feature with the smallest difference is __*industry*__.

### Question 6

   * Now let's train a regularized logistic regression.
   * Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
   * Train models using all the features as in Q4.
   * Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

   * 0.01
   * 0.1
   * 1
   * 10
   * 100

    Note: If there are multiple options, select the smallest C.



In [64]:
c_list = [0.01, 0.1, 1, 10, 100]

for c in c_list:
    #additional _reg for regularized
    model_reg = LogisticRegression(penalty='l2', solver='liblinear', C=c, max_iter=1000, random_state=42) #penalty l1 Lasso, l2 Ridge
    model_reg.fit(X_train, y_train)
    y_pred_reg = model_reg.predict_proba(X_val)[:,1]
    converted_decision_reg = (y_pred_reg >= 0.5)
    accuracy_reg = round((y_val == converted_decision_reg).mean(), 3)


    print(f'Accuracy of "C = {c}" on validation dataset is: {accuracy_reg}')
    


Accuracy of "C = 0.01" on validation dataset is: 0.7
Accuracy of "C = 0.1" on validation dataset is: 0.7
Accuracy of "C = 1" on validation dataset is: 0.7
Accuracy of "C = 10" on validation dataset is: 0.7
Accuracy of "C = 100" on validation dataset is: 0.7


**Answer**: since all c in c_list give same accuracy and following the clue in the question, then we choose ***0.01*** as the best accuracy.